# 安装相关套件
## Step1.安装openslide (使用国内镜像加速)
pip install openslide-bin -i https://pypi.tuna.tsinghua.edu.cn/simple/

## Step2.安装pyhist
git clone https://github.com/baidu-research/pyhist.git  
cd PyHIST  
pip install -e .  

## Step3.下载一张svs格式的病理切片
素材源于nih https://brd.nci.nih.gov/brd/specimen/GTEX-1117F-0126

## 在这只要安装第一步即可，pyhist已经置入本专案中。

##### 打印层级信息  
金字塔状

In [ ]:
import os, glob
import openslide

# 假设当前目录下有 *.svs
svs_path = glob.glob('data/*.svs')[0]   # 自动取第一个文件
print("使用文件：", svs_path)

slide = openslide.OpenSlide(svs_path)
print("层级数量 :", slide.level_count)
for l in range(slide.level_count):
    dims  = slide.level_dimensions[l]
    down  = slide.level_downsamples[l]
    print(f"层级 {l}: 尺寸{dims}, 下采样因子{down:.2f}")

##### 使用PyHist进行切图

In [ ]:
from src.slide import PySlide, TileGenerator
import argparse
import os, sys
sys.path.insert(0, os.path.abspath("PyHIST"))

# 2. CLI 等价参数
args = argparse.Namespace(
    # 基本 IO
    svs=svs_path,
    output="out",                     # 输出目录
    format="png",                     # patch 格式
    info="default",                   # 日志级别

    # 切图参数
    patch_size=256,
    output_downsample=2,              # 20×
    mask_downsample=16,               # 掩膜降采样
    tilecross_downsample=16,          # 预览图降采样
    content_threshold=0.3,            # 前景占比阈值
    thres=0.3, 
    method="otsu",                    # 无需编译，直接可用

    # 保存控制
    save_patches=True,
    save_tilecrossed_image=True,
    save_mask=True,
    save_blank=False,
    save_nonsquare=False,

    # graph 专用（method 非 graph 时可忽略）
    k_const=10000,
    minimum_segmentsize=10000,
    percentage_bc=5,
    sigma=0.5,
    borders="1111",
    corners="0000",
    save_edges=False,

    # random sampling 专用（method 非 randomsampling 时可忽略）
    npatches=100,
)

# 2. 转成字典再传入
slide = PySlide(vars(args))   # ✅ vars(ns) 返回 dict
extractor = TileGenerator(slide)
extractor.execute()